# Lab 2: Text Generation Experiments

**Week 1 - GenAI Introduction & Fundamentals**

**Provided by:** ADC ENGINEERING & CONSULTING LTD

## Objectives

In this lab, you will:
- Understand the difference between completion and chat models
- Experiment with various text generation parameters
- Build a simple chatbot with conversation memory
- Implement token counting and cost tracking
- Explore different text generation use cases
- Create streaming responses for better UX

## Prerequisites

- Completed Lab 1
- OpenAI API key configured
- Python 3.9+
- Basic understanding of API parameters

## Setup and Installation

In [ ]:
# Install required packages
!pip install openai python-dotenv tiktoken --quiet

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import tiktoken
import time
from typing import List, Dict

# Load environment variables
load_dotenv()

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

print("✓ Setup complete!")

## Part 1: Completion vs Chat Models

OpenAI offers two types of models:
- **Completion models**: Continue text from a prompt (legacy)
- **Chat models**: Conversational, with roles (system, user, assistant)

Modern applications primarily use **chat models** (gpt-3.5-turbo, gpt-4).

### Chat Model Architecture

Chat models use a message-based format with three roles:
- **system**: Sets behavior and instructions
- **user**: The human's input
- **assistant**: The AI's responses

In [ ]:
# Example: Simple chat interaction
def simple_chat(user_message: str) -> str:
    """Basic chat completion."""
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": user_message}
        ]
    )
    
    return response.choices[0].message.content

# Test it
result = simple_chat("What are the three laws of robotics?")
print(result)

### Exercise 1.1: Understanding Message Roles

Create a chat function that demonstrates how different system messages change behavior:

In [ ]:
def chat_with_persona(user_message: str, persona: str) -> str:
    """
    Chat with different personas.
    
    Args:
        user_message: The user's question
        persona: One of 'formal', 'casual', 'technical', 'creative'
    
    Returns:
        The assistant's response
    """
    
    personas = {
        'formal': "You are a formal business consultant. Use professional language and structure.",
        'casual': "You are a friendly buddy. Use casual language, emojis, and be conversational.",
        'technical': "You are a senior engineer. Provide technical, detailed explanations with examples.",
        'creative': "You are a creative writer. Use vivid imagery, metaphors, and engaging narratives."
    }
    
    # TODO: Implement this function
    # Hint: Use the personas dictionary to get the system message
    
    pass

# Test different personas
question = "Explain what cloud computing is."

for persona in ['formal', 'casual', 'technical', 'creative']:
    print(f"\n{persona.upper()} PERSONA:")
    print("-" * 80)
    # TODO: Call chat_with_persona and print the result
    # response = chat_with_persona(question, persona)
    # print(response)
    print()

## Part 2: Advanced Parameter Exploration

Let's explore parameters beyond temperature and max_tokens.

### Top P (Nucleus Sampling)

An alternative to temperature that controls randomness by selecting from the top tokens that sum to probability P.

- **top_p = 0.1**: Very focused, only top 10% probability tokens
- **top_p = 0.9**: Balanced (default)
- **top_p = 1.0**: Consider all tokens

In [ ]:
def explore_top_p(prompt: str, top_p_values: List[float] = [0.1, 0.5, 0.9]):
    """Experiment with top_p sampling."""
    
    print(f"Prompt: {prompt}\n")
    print("=" * 80)
    
    for top_p in top_p_values:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=1.0,  # High temperature to see top_p effect
            top_p=top_p,
            max_tokens=100
        )
        
        print(f"\nTop P: {top_p}")
        print("-" * 80)
        print(response.choices[0].message.content)
        print("=" * 80)

# Test with a creative prompt
creative_prompt = "Complete this story: 'The door creaked open, revealing...'"
explore_top_p(creative_prompt)

### Frequency and Presence Penalties

Control repetition in generated text:
- **frequency_penalty** (-2.0 to 2.0): Penalize tokens based on how often they appear
- **presence_penalty** (-2.0 to 2.0): Penalize tokens that have appeared at all

In [ ]:
def explore_penalties(prompt: str):
    """Test frequency and presence penalties."""
    
    configs = [
        {"name": "No penalties", "frequency": 0.0, "presence": 0.0},
        {"name": "High frequency penalty", "frequency": 2.0, "presence": 0.0},
        {"name": "High presence penalty", "frequency": 0.0, "presence": 2.0},
        {"name": "Both high", "frequency": 1.5, "presence": 1.5},
    ]
    
    print(f"Prompt: {prompt}\n")
    
    for config in configs:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            frequency_penalty=config["frequency"],
            presence_penalty=config["presence"],
            max_tokens=150,
            temperature=0.7
        )
        
        print(f"\n{config['name']}")
        print(f"(frequency={config['frequency']}, presence={config['presence']})")
        print("-" * 80)
        print(response.choices[0].message.content)
        print("=" * 80)

# Test with a prompt that tends to be repetitive
repetitive_prompt = "List 10 benefits of exercise."
explore_penalties(repetitive_prompt)

### Exercise 2.1: Parameter Tuning Challenge

For each use case, determine the optimal parameters:

In [ ]:
# Use cases and their optimal parameters

use_cases = {
    "legal_document": {
        "prompt": "Draft a privacy policy section about data collection.",
        "temperature": 0.0,  # TODO: Adjust
        "top_p": 1.0,  # TODO: Adjust
        "frequency_penalty": 0.0,  # TODO: Adjust
        "presence_penalty": 0.0  # TODO: Adjust
    },
    "creative_story": {
        "prompt": "Write an opening paragraph for a mystery novel.",
        "temperature": 0.0,  # TODO: Adjust
        "top_p": 1.0,  # TODO: Adjust
        "frequency_penalty": 0.0,  # TODO: Adjust
        "presence_penalty": 0.0  # TODO: Adjust
    },
    "technical_docs": {
        "prompt": "Explain how to implement binary search in Python.",
        "temperature": 0.0,  # TODO: Adjust
        "top_p": 1.0,  # TODO: Adjust
        "frequency_penalty": 0.0,  # TODO: Adjust
        "presence_penalty": 0.0  # TODO: Adjust
    },
    "marketing_copy": {
        "prompt": "Write 5 taglines for an eco-friendly water bottle.",
        "temperature": 0.0,  # TODO: Adjust
        "top_p": 1.0,  # TODO: Adjust
        "frequency_penalty": 0.0,  # TODO: Adjust
        "presence_penalty": 0.0  # TODO: Adjust
    }
}

# Test your parameter choices
for use_case, config in use_cases.items():
    print(f"\n{'='*80}")
    print(f"USE CASE: {use_case.upper()}")
    print(f"{'='*80}\n")
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": config["prompt"]}],
        temperature=config["temperature"],
        top_p=config["top_p"],
        frequency_penalty=config["frequency_penalty"],
        presence_penalty=config["presence_penalty"],
        max_tokens=200
    )
    
    print(f"Prompt: {config['prompt']}")
    print(f"\nParameters:")
    print(f"  Temperature: {config['temperature']}")
    print(f"  Top P: {config['top_p']}")
    print(f"  Frequency Penalty: {config['frequency_penalty']}")
    print(f"  Presence Penalty: {config['presence_penalty']}")
    print(f"\nResponse:")
    print(response.choices[0].message.content)
    print()

## Part 3: Building a Chatbot with Memory

Create a conversational chatbot that remembers context.

In [ ]:
class Chatbot:
    """
    A simple chatbot with conversation memory.
    """
    
    def __init__(self, system_message: str = "You are a helpful assistant.", model: str = "gpt-3.5-turbo"):
        """
        Initialize the chatbot.
        
        Args:
            system_message: The system prompt defining behavior
            model: The OpenAI model to use
        """
        self.model = model
        self.messages = [
            {"role": "system", "content": system_message}
        ]
        self.total_tokens = 0
    
    def chat(self, user_message: str) -> str:
        """
        Send a message and get a response.
        
        Args:
            user_message: The user's message
        
        Returns:
            The assistant's response
        """
        # Add user message to history
        self.messages.append({"role": "user", "content": user_message})
        
        # Get response
        response = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            temperature=0.7
        )
        
        # Extract assistant's message
        assistant_message = response.choices[0].message.content
        
        # Add to history
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        # Track tokens
        self.total_tokens += response.usage.total_tokens
        
        return assistant_message
    
    def get_conversation(self) -> List[Dict[str, str]]:
        """Get the full conversation history."""
        return self.messages[1:]  # Exclude system message
    
    def get_token_usage(self) -> int:
        """Get total tokens used."""
        return self.total_tokens
    
    def clear_history(self):
        """Clear conversation history (keep system message)."""
        self.messages = self.messages[:1]
        self.total_tokens = 0

# Test the chatbot
bot = Chatbot(system_message="You are a friendly Python tutor.")

print("Chatbot initialized!\n")

# Conversation 1
print("User: What is a list in Python?")
response1 = bot.chat("What is a list in Python?")
print(f"Assistant: {response1}\n")

# Conversation 2 (should remember context)
print("User: Can you give me an example?")
response2 = bot.chat("Can you give me an example?")
print(f"Assistant: {response2}\n")

# Conversation 3 (testing memory)
print("User: What was my first question?")
response3 = bot.chat("What was my first question?")
print(f"Assistant: {response3}\n")

print(f"Total tokens used: {bot.get_token_usage()}")

### Exercise 3.1: Enhanced Chatbot

Enhance the Chatbot class with:
1. Token limit management
2. Automatic summarization when approaching limits
3. Export conversation to file
4. Load conversation from file

In [ ]:
class EnhancedChatbot(Chatbot):
    """
    Enhanced chatbot with token management and persistence.
    """
    
    def __init__(self, system_message: str = "You are a helpful assistant.", 
                 model: str = "gpt-3.5-turbo", max_tokens: int = 4000):
        super().__init__(system_message, model)
        self.max_tokens = max_tokens
    
    def get_current_tokens(self) -> int:
        """
        Count tokens in current conversation.
        
        TODO: Implement using tiktoken
        Hint: Count tokens in all messages
        """
        pass
    
    def summarize_conversation(self) -> str:
        """
        Summarize the conversation to reduce token count.
        
        TODO: Implement
        Hint: Ask the model to summarize previous messages
        """
        pass
    
    def check_and_manage_tokens(self):
        """
        Check token count and summarize if needed.
        
        TODO: Implement
        Hint: If current_tokens > 80% of max_tokens, summarize
        """
        pass
    
    def save_conversation(self, filepath: str):
        """
        Save conversation to JSON file.
        
        TODO: Implement
        Hint: Use json.dump()
        """
        pass
    
    def load_conversation(self, filepath: str):
        """
        Load conversation from JSON file.
        
        TODO: Implement
        Hint: Use json.load()
        """
        pass

# Test your implementation
# bot = EnhancedChatbot(max_tokens=500)
# response = bot.chat("Tell me about Python.")
# print(f"Current tokens: {bot.get_current_tokens()}")

## Part 4: Token Counting and Cost Tracking

Implement comprehensive token counting and cost estimation.

In [ ]:
class TokenCounter:
    """
    Utility class for token counting and cost estimation.
    """
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.encoding = tiktoken.encoding_for_model(model)
    
    def count_tokens(self, text: str) -> int:
        """Count tokens in a text string."""
        return len(self.encoding.encode(text))
    
    def count_message_tokens(self, messages: List[Dict[str, str]]) -> int:
        """
        Count tokens in a list of messages.
        
        Note: This is an approximation. The actual count may vary slightly.
        """
        tokens = 0
        
        for message in messages:
            # Every message has some overhead
            tokens += 4  # Message formatting tokens
            
            for key, value in message.items():
                tokens += self.count_tokens(value)
        
        tokens += 2  # Conversation priming tokens
        
        return tokens
    
    def estimate_cost(self, input_tokens: int, output_tokens: int) -> Dict[str, float]:
        """
        Estimate cost based on token usage.
        
        Pricing (as of 2024):
        - gpt-3.5-turbo: $0.0005/1K input, $0.0015/1K output
        - gpt-4: $0.03/1K input, $0.06/1K output
        """
        
        pricing = {
            "gpt-3.5-turbo": {"input": 0.0005, "output": 0.0015},
            "gpt-4": {"input": 0.03, "output": 0.06},
            "gpt-4-turbo": {"input": 0.01, "output": 0.03}
        }
        
        if self.model not in pricing:
            return {"error": f"Pricing not available for {self.model}"}
        
        input_cost = (input_tokens / 1000) * pricing[self.model]["input"]
        output_cost = (output_tokens / 1000) * pricing[self.model]["output"]
        
        return {
            "input_tokens": input_tokens,
            "output_tokens": output_tokens,
            "total_tokens": input_tokens + output_tokens,
            "input_cost": input_cost,
            "output_cost": output_cost,
            "total_cost": input_cost + output_cost
        }

# Test token counter
counter = TokenCounter("gpt-3.5-turbo")

# Count tokens in a message
text = "Hello, how are you doing today?"
token_count = counter.count_tokens(text)
print(f"Text: {text}")
print(f"Tokens: {token_count}\n")

# Count tokens in messages
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is machine learning?"},
    {"role": "assistant", "content": "Machine learning is a subset of artificial intelligence..."}
]

message_tokens = counter.count_message_tokens(messages)
print(f"Message tokens: {message_tokens}\n")

# Estimate cost
cost_info = counter.estimate_cost(input_tokens=100, output_tokens=200)
print("Cost estimation:")
for key, value in cost_info.items():
    if 'cost' in key:
        print(f"  {key}: ${value:.6f}")
    else:
        print(f"  {key}: {value}")

### Exercise 4.1: Cost-Aware Chatbot

Create a chatbot that tracks and displays costs in real-time:

In [ ]:
class CostAwareChatbot:
    """
    Chatbot with real-time cost tracking.
    
    TODO: Implement a chatbot that:
    1. Tracks tokens and costs for each message
    2. Shows cumulative costs
    3. Warns when costs exceed a threshold
    4. Generates cost reports
    """
    
    def __init__(self, system_message: str, model: str = "gpt-3.5-turbo", 
                 cost_threshold: float = 0.10):
        self.model = model
        self.messages = [{"role": "system", "content": system_message}]
        self.cost_threshold = cost_threshold
        self.total_cost = 0.0
        self.conversation_history = []  # Track each exchange
        self.counter = TokenCounter(model)
    
    # TODO: Implement methods:
    # - chat(user_message) -> response with cost info
    # - get_total_cost() -> total cost so far
    # - get_cost_report() -> detailed cost breakdown
    # - check_threshold() -> warn if threshold exceeded
    
    pass

# Test
# bot = CostAwareChatbot("You are a helpful assistant.", cost_threshold=0.01)
# response = bot.chat("Explain quantum computing.")
# print(f"Total cost so far: ${bot.get_total_cost():.6f}")

## Part 5: Streaming Responses

Implement streaming for better user experience with long responses.

In [ ]:
def stream_chat(user_message: str, system_message: str = "You are a helpful assistant."):
    """
    Stream a chat response token by token.
    """
    
    print("Assistant: ", end="", flush=True)
    
    stream = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_message}
        ],
        stream=True,
        temperature=0.7
    )
    
    full_response = ""
    
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            content = chunk.choices[0].delta.content
            print(content, end="", flush=True)
            full_response += content
    
    print("\n")  # New line at the end
    
    return full_response

# Test streaming
print("User: Write a short poem about coding.\n")
response = stream_chat("Write a short poem about coding.")

In [ ]:
class StreamingChatbot:
    """
    Chatbot with streaming support.
    """
    
    def __init__(self, system_message: str = "You are a helpful assistant.", model: str = "gpt-3.5-turbo"):
        self.model = model
        self.messages = [{"role": "system", "content": system_message}]
    
    def chat_stream(self, user_message: str):
        """
        Stream a response and yield chunks.
        """
        # Add user message
        self.messages.append({"role": "user", "content": user_message})
        
        # Create streaming response
        stream = client.chat.completions.create(
            model=self.model,
            messages=self.messages,
            stream=True,
            temperature=0.7
        )
        
        full_response = ""
        
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                full_response += content
                yield content
        
        # Add complete response to history
        self.messages.append({"role": "assistant", "content": full_response})

# Test streaming chatbot
bot = StreamingChatbot("You are a creative storyteller.")

print("User: Tell me a very short story about a robot.\n")
print("Assistant: ", end="", flush=True)

for chunk in bot.chat_stream("Tell me a very short story about a robot."):
    print(chunk, end="", flush=True)

print("\n")

### Exercise 5.1: Streaming with Indicators

Enhance the streaming chatbot to show:
1. Typing indicator while waiting
2. Token count updates in real-time
3. Estimated cost as response generates

In [ ]:
import sys

class EnhancedStreamingChatbot(StreamingChatbot):
    """
    Streaming chatbot with visual indicators.
    
    TODO: Implement enhanced streaming features:
    1. Show "Thinking..." before response starts
    2. Display token count as response generates
    3. Show estimated cost in real-time
    4. Add progress indicator
    """
    
    def __init__(self, system_message: str = "You are a helpful assistant.", 
                 model: str = "gpt-3.5-turbo"):
        super().__init__(system_message, model)
        self.counter = TokenCounter(model)
    
    # TODO: Implement enhanced streaming methods
    
    pass

# Test your implementation
# bot = EnhancedStreamingChatbot()
# bot.chat_stream_with_indicators("Explain how neural networks work.")

## Part 6: Text Generation Use Cases

Explore different text generation applications.

### Use Case 1: Text Summarization

In [ ]:
def summarize_text(text: str, style: str = "concise") -> str:
    """
    Summarize a text with different styles.
    
    Args:
        text: The text to summarize
        style: 'concise', 'bullet', 'executive', 'eli5'
    """
    
    styles = {
        'concise': "Summarize the following text in 2-3 sentences.",
        'bullet': "Summarize the following text as bullet points highlighting key information.",
        'executive': "Provide an executive summary suitable for a business audience.",
        'eli5': "Explain the following text like I'm 5 years old."
    }
    
    prompt = f"{styles.get(style, styles['concise'])}\n\nText: {text}"
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=300
    )
    
    return response.choices[0].message.content

# Test summarization
long_text = """
Artificial intelligence (AI) is transforming the modern world in unprecedented ways. 
From healthcare to finance, transportation to entertainment, AI systems are being 
deployed to solve complex problems and enhance human capabilities. Machine learning, 
a subset of AI, enables computers to learn from data without explicit programming. 
Deep learning, which uses neural networks with multiple layers, has achieved 
remarkable results in image recognition, natural language processing, and game playing. 
However, the rapid advancement of AI also raises important ethical questions about 
privacy, bias, job displacement, and the future of human-AI interaction. As we continue 
to develop more sophisticated AI systems, it's crucial to ensure they are designed 
with human values in mind and remain beneficial to society.
"""

print("Original text length:", len(long_text), "characters\n")

for style in ['concise', 'bullet', 'executive', 'eli5']:
    print(f"\n{style.upper()} SUMMARY:")
    print("-" * 80)
    summary = summarize_text(long_text, style)
    print(summary)
    print(f"Summary length: {len(summary)} characters")
    print("=" * 80)

### Use Case 2: Content Generation

In [ ]:
def generate_content(content_type: str, topic: str, **kwargs) -> str:
    """
    Generate different types of content.
    
    Args:
        content_type: 'blog', 'email', 'social', 'ad_copy'
        topic: The subject matter
        **kwargs: Additional parameters (tone, length, etc.)
    """
    
    prompts = {
        'blog': f"Write a {kwargs.get('length', 'short')} blog post about {topic}. Tone: {kwargs.get('tone', 'professional')}.",
        'email': f"Write a {kwargs.get('tone', 'professional')} email about {topic}.",
        'social': f"Write a {kwargs.get('platform', 'Twitter')} post about {topic}. Keep it engaging and concise.",
        'ad_copy': f"Write compelling ad copy for {topic}. Focus on benefits and include a call-to-action."
    }
    
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompts[content_type]}],
        temperature=0.8,
        max_tokens=kwargs.get('max_tokens', 300)
    )
    
    return response.choices[0].message.content

# Test content generation
print("BLOG POST:")
print("=" * 80)
blog = generate_content('blog', 'benefits of remote work', length='short', tone='professional')
print(blog)
print("\n" + "=" * 80 + "\n")

print("EMAIL:")
print("=" * 80)
email = generate_content('email', 'quarterly meeting invitation', tone='friendly')
print(email)
print("\n" + "=" * 80 + "\n")

print("SOCIAL MEDIA POST:")
print("=" * 80)
social = generate_content('social', 'new product launch', platform='LinkedIn')
print(social)

### Exercise 6.1: Multi-Purpose Text Generator

Create a comprehensive text generation tool that can:
1. Generate content for multiple use cases
2. Apply different tones and styles
3. Track token usage and costs
4. Save generated content to files

In [ ]:
class TextGenerator:
    """
    Multi-purpose text generation tool.
    
    TODO: Implement a text generator with:
    1. Multiple content types (blog, email, social, docs, etc.)
    2. Customizable parameters (tone, length, style)
    3. Cost tracking
    4. Export functionality
    5. Batch generation
    """
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.counter = TokenCounter(model)
        self.generation_history = []
    
    # TODO: Implement methods
    
    pass

# Test your implementation
# generator = TextGenerator()
# content = generator.generate('blog', 'AI in healthcare', tone='professional', length='medium')
# print(content)

## Challenge Projects

### Challenge 1: Interactive Story Generator

Build an interactive story generator where:
- User makes choices that affect the story
- AI generates story segments based on choices
- Track multiple story branches
- Save/load story progress

In [ ]:
class InteractiveStory:
    """
    Interactive story generator with branching narratives.
    
    TODO: Implement:
    1. Story initialization with setting/genre
    2. Generate story segments
    3. Present choices to the user
    4. Branch based on user decisions
    5. Track story state and history
    6. Save/load functionality
    """
    
    def __init__(self, genre: str, setting: str):
        self.genre = genre
        self.setting = setting
        self.story_segments = []
        self.choices_made = []
    
    def start_story(self):
        """Generate opening segment."""
        pass
    
    def generate_choices(self, current_segment: str) -> List[str]:
        """Generate 2-3 choices for the user."""
        pass
    
    def continue_story(self, choice: str):
        """Generate next segment based on choice."""
        pass
    
    def get_full_story(self) -> str:
        """Compile all segments into full story."""
        pass

# Usage example:
# story = InteractiveStory(genre="sci-fi", setting="space station")
# story.start_story()
# while not story.is_complete():
#     choices = story.generate_choices(story.current_segment)
#     user_choice = input(f"Choose {choices}: ")
#     story.continue_story(user_choice)

### Challenge 2: Multi-Language Translation Service

Create a translation service that:
- Translates text to multiple languages
- Maintains context and tone
- Handles idioms and cultural nuances
- Provides back-translation verification
- Estimates costs for batch translations

In [ ]:
class TranslationService:
    """
    Multi-language translation service.
    
    TODO: Implement:
    1. Translate text to specified language
    2. Batch translation
    3. Context preservation
    4. Back-translation for verification
    5. Cost estimation
    6. Quality scoring
    """
    
    def __init__(self, model: str = "gpt-3.5-turbo"):
        self.model = model
        self.supported_languages = [
            'Spanish', 'French', 'German', 'Italian', 'Portuguese',
            'Chinese', 'Japanese', 'Korean', 'Arabic', 'Russian'
        ]
    
    # TODO: Implement translation methods
    
    pass

# Usage example:
# translator = TranslationService()
# result = translator.translate("Hello, how are you?", target_language="Spanish")
# print(result)

### Challenge 3: Content Repurposing Tool

Build a tool that repurposes content across different formats:
- Blog post → Social media posts
- Article → Email newsletter
- Long-form → Multiple short-form pieces
- Technical docs → User-friendly guides

In [ ]:
class ContentRepurposer:
    """
    Repurpose content across different formats and platforms.
    
    TODO: Implement:
    1. Analyze input content
    2. Convert to different formats
    3. Optimize for each platform
    4. Maintain key messages
    5. Generate multiple variations
    6. Track conversion quality
    """
    
    def __init__(self):
        self.formats = {
            'twitter': {'max_length': 280, 'style': 'concise'},
            'linkedin': {'max_length': 1300, 'style': 'professional'},
            'email': {'max_length': 500, 'style': 'conversational'},
            'instagram': {'max_length': 2200, 'style': 'visual'}
        }
    
    # TODO: Implement repurposing methods
    
    pass

# Usage example:
# repurposer = ContentRepurposer()
# original = "Long blog post content..."
# social_posts = repurposer.convert_to_social(original, platforms=['twitter', 'linkedin'])

## Summary

In this lab, you've learned:

1. ✅ Differences between completion and chat models
2. ✅ Advanced parameter tuning (temperature, top_p, penalties)
3. ✅ Building chatbots with conversation memory
4. ✅ Token counting and cost estimation
5. ✅ Implementing streaming responses
6. ✅ Various text generation use cases

### Key Takeaways

- **Chat models** are the modern standard for most applications
- **Parameters matter**: Different tasks require different settings
- **Memory management**: Critical for maintaining context in conversations
- **Cost awareness**: Always track token usage and costs
- **Streaming**: Improves UX for long-form generation
- **Use case specific**: Tailor prompts and parameters to your application

### Best Practices

1. **Start with low temperature** for consistent results, increase for creativity
2. **Use presence_penalty** to encourage topic diversity
3. **Implement token limits** to prevent runaway costs
4. **Stream responses** for better user experience
5. **Track costs** in production applications
6. **Test parameters** systematically for each use case

### Next Steps

- Complete the challenge projects
- Build your own text generation application
- Experiment with different model combinations
- Move on to Lab 3: Building a Simple AI Application

**Provided by:** ADC ENGINEERING & CONSULTING LTD